<a href="https://colab.research.google.com/github/fitditcorp4/Python-Projects/blob/main/weekly_dataset02022023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
# imports library
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.plotting import autocorrelation_plot
from random import randrange
from statsmodels.tsa.seasonal import seasonal_decompose
#from pandas import datetime
from datetime import datetime
import numpy as np
from pandas import read_json
from pandas import read_csv
from numpy import array
import matplotlib.colors as colors
import matplotlib.cm as cm
from pylab import rcParams
import warnings
import itertools
from pandas import Series
# plt.style.use('fivethirtyeight')
plt.style.use('classic')
%matplotlib inline

In [72]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 1000)

In [73]:
from google.colab import data_table
data_table.enable_dataframe_formatter

<function google.colab.data_table.enable_dataframe_formatter()>

In [74]:
df0 = pd.read_json('https://oilspillmonitor.ng/api/spill-data.php?dataset=nosdra&format=json')

In [75]:
 df0

,id,status,company,incidentnumber,incidentdate,contaminant,estimatedquantity,sitelocationname,spillareahabitat,lastupdatedby,cause,updatefor,latitude,longitude,statesaffected,attachments,jivdate,lga,zonaloffice,certificatedate,certificatenumber,formadate,reportdate,spillstopdate,initialcontainmentmeasures,descriptionofimpact,formbdate,formcdate,jivpresent,typeoffacility,cleanupcompleteddate,cleanupmethods,cleanupdate,quantityrecovered,postcleanupinspectiondate,finalsamplingdate,estimatedspillarea,remediationstart,remediationtype,remediationend,postimpactassessmentdate,finallabresultsdate
0,2,confirmed,ADDAX,HSE/OBO/0611/101,2006-11-23,cr,225,Subsea Pipeline @ Mimbo Platform (OML123),of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,confirmed,ADDAX,HSE/OBO/0612/108,2006-12-18,cr,0.5,EBNE04HT @ Ebughu Platform (OML 123),of,NOSDRA,eqf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,confirmed,ADDAX,HSE/OBO/0612/110,2006-12-27,cr,25,South East of BOGI Platform,of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,confirmed,ADDAX,HSE/OBO/0706/166,2007-05-14,cr,2,Adanga Platform,of,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,confirmed,ADDAX,HSE/OBO/0708/201,2007-08-16,gs,NaN,"Izombe Well #10LS 4"" Flowline",la,NOSDRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16418,268092,confirmed,Midwestern,UPIL/001/2023,2023-01-23,cr,0.5,12'' MIDWESTERN OIL AND GAS PIPELINE AT EBENDO,la,2023-01-28 WA003,sab,NaN,5.6259517,6.3087708,DE,"[{""url"":""data/attachments/t185f81a4698/1.jpg"",...",2023-01-26,Isoko-North,wa,NaN,NaN,NaN,2023-01-23,NaN,other:none,"Soil was impacted, the spill was within the co...",NaN,NaN,"nosdra,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16419,268093,confirmed,SPDC,828857,2023-01-07,no,NaN,4'' YOKRI WELL 113T FLOWLINE (LOCATION 2) AT O...,la,2023-01-28 WA003,sab,NaN,5.4198700,5.2971800,DE,"[{""url"":""data/attachments/t185f821fa19/1.jpg"",...",2023-01-25,Burutu,wa,NaN,NaN,NaN,2023-01-07,NaN,NaN,An illegal 2'' ball valve was installed on the...,NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16420,268109,confirmed,SPDC,2023_864732,2023-01-24,cr,NaN,20'' Kolo Creek Traans-Niger Pipeline at Ihuow...,la,2023-02-02 PH021,sab,268109.0,4.9750550,6.6635089,RI,"[{""url"":""data/attachments/268109/IMG_20230125_...",2023-01-25,Ahoada-East,ph,NaN,NaN,NaN,2023-01-24,NaN,NaN,No impact,NaN,NaN,"cmmnt,cmpny,mnstry,nosdra,plc",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16421,268111,confirmed,SEPLAT,SEPLAT/WA/OS/13H/2022,2022-09-05,cr,20,20'' AMUKPE - ESCRAVOS PIPELINE AT KM 51,sw,2023-02-02 WA003,sab,NaN,5.7119722,5.2638333,DE,"[{""url"":""data/attachments/t186123fa452/1.jpg"",...",2023-01-16,Warri-South-West,wa,NaN,NaN,NaN,2022-09-06,NaN,bm,"Fishpond, vegetation, surface water and soil w...",NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df1 = df0.loc[df0['status'].isin(['confirmed', 'reviewed'])]

In [93]:
#df1['incidentdate'].value_counts()

In [78]:
df1.replace(to_replace='20014-07-08', value='2014-07-08',inplace=True)  # Replaces a typograpical error
df1.replace(to_replace='2016-01-13 AB008', value='2016-01-13',inplace=True)
df1.replace(to_replace='2016-04-14 AB014', value='2016-01-13',inplace=True)
df1.replace(to_replace='2019-08-19 PH133', value='2019-08-19',inplace=True)
df1.replace(to_replace='2019-08-19 PH133, 2020-12-15 PH030, 2021-09-17 PH012, 2021-09-20 PH012, 2021-10-31 PH012', value='2019-08-19',inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [79]:
df1.incidentdate.isna().sum()

590

In [80]:
df1.loc[pd.isnull(df1['incidentdate']), 'incidentdate'] = df1.reportdate
df1.loc[pd.isnull(df1['incidentdate']), 'incidentdate'] = df1.formadate
df1.loc[pd.isnull(df1['incidentdate']), 'incidentdate'] = df1.jivdate
df1.loc[pd.isnull(df1['incidentdate']), 'incidentdate'] = df1.lastupdatedby

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [81]:
df1.incidentdate.isna().sum()

0

In [82]:
df1['contaminant'] = df1['contaminant'].fillna('notavailable')
df1.loc[df1['contaminant'].str.startswith('other'), 'contaminant'] = 'other'
df1.loc[df1['contaminant'].str.startswith('oth'), 'contaminant'] = 'other'
df1.loc[df1['contaminant'].str.startswith('re'), 'contaminant'] = 're'
df1.loc[df1['contaminant'].str.startswith('Die'), 'contaminant'] = 're'
df1.loc[df1['contaminant'].str.startswith('not'), 'contaminant'] = 'na'
df1.loc[df1['contaminant'].str.startswith('Con'), 'contaminant'] = 'co'
df1.loc[df1['contaminant'].str.startswith('CON'), 'contaminant'] = 'co'
df1.loc[df1['contaminant'].str.startswith('co'), 'contaminant'] = 'co'
df1.loc[df1['contaminant'].str.startswith('NIL'), 'contaminant'] = 'na'
df1.loc[df1['contaminant'].str.startswith('LPFO'), 'contaminant'] = 're'
df1.loc[df1['contaminant'].str.startswith('Pro'), 'contaminant'] = 'other'
df1.loc[df1['contaminant'].str.startswith('sheen'), 'contaminant'] = 'other'
df1.loc[df1['contaminant'].str.startswith('Ass'), 'contaminant'] = 'other'
df1.loc[df1['contaminant'].str.startswith('gas'), 'contaminant'] = 'ga'
df1.loc[df1['contaminant'].str.startswith('wa'), 'contaminant'] = 'na'
df1.loc[df1['contaminant'].str.startswith('F'), 'contaminant'] = 'na'
df1.loc[df1['contaminant'].str.startswith('0'), 'contaminant'] = 'na'
df1.loc[df1['contaminant'].str.startswith('gs'), 'contaminant'] = 'ga'
df1.loc[df1['contaminant'].str.startswith('Crude'), 'contaminant'] = 'cr'
df1.loc[df1['contaminant'].str.startswith('crude'), 'contaminant'] = 'cr'

<ipython-input-82-25e6b16a420f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['contaminant'] = df1['contaminant'].fillna('notavailable')
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [83]:
df1.loc[df1['contaminant'].str.startswith('GAS'), 'contaminant'] = 'ga'
df1.loc[df1['contaminant'].str.startswith('None'), 'contaminant'] = 'no'

In [84]:
df1.contaminant.unique()

array(['cr', 'ga', 'other', 'na', 'ch', 're', ' ', 'co', 'no',
       'Non-Leaking', 'Non leaking'], dtype=object)

In [85]:
df2 = df1.loc[df1['contaminant'].isin(['cr', 're', 'co', 'ch'])]

In [86]:
df2.estimatedquantity.isna().sum()

2340

In [ ]:
df2.loc[df2['estimatedquantity'].str.startswith('0', na=False), 'estimatedquantity'] = df2['estimatedquantity'].replace(',', '.',  regex=True)
df2['estimatedquantity'] = df2['estimatedquantity'].replace(',', '',  regex=True)
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('bbl', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BBL', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Bbl', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Barrel', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BARREL', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('bblS', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BBLS', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('barrels', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Barrels', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('I', '1')

In [ ]:
df2.loc[df2['estimatedquantity'].str.startswith('0', na=False), 'estimatedquantity'] = df2['estimatedquantity'].replace(',', '.',  regex=True)
df2['estimatedquantity'] = df2['estimatedquantity'].replace(',', '',  regex=True)
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('bbl', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BBL', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Bbl', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Barrel', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BARREL', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('bblS', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('BBLS', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('barrels', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('Barrels', '')
df2['estimatedquantity'] = df2['estimatedquantity'].str.replace('I', '1')

In [88]:
df3 = df2.loc[(df2['incidentdate'] > '2015-12-31') & (df2['incidentdate'] < '2023-01-01')]

In [89]:
df3['incidentdate'] = pd.DatetimeIndex(df3['incidentdate'])

<ipython-input-89-3ce1f0b5bdb3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['incidentdate'] = pd.DatetimeIndex(df3['incidentdate'])


In [90]:
df3.estimatedquantity = pd.to_numeric(df3.estimatedquantity, errors = 'coerce')

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [97]:
df3.columns

Index(['id', 'status', 'company', 'incidentnumber', 'incidentdate', 'contaminant', 'estimatedquantity', 'sitelocationname', 'spillareahabitat', 'lastupdatedby', 'cause', 'updatefor', 'latitude', 'longitude', 'statesaffected', 'attachments', 'jivdate', 'lga', 'zonaloffice', 'certificatedate', 'certificatenumber', 'formadate', 'reportdate', 'spillstopdate', 'initialcontainmentmeasures', 'descriptionofimpact', 'formbdate', 'formcdate', 'jivpresent', 'typeoffacility', 'cleanupcompleteddate', 'cleanupmethods', 'cleanupdate', 'quantityrecovered', 'postcleanupinspectiondate', 'finalsamplingdate', 'estimatedspillarea', 'remediationstart', 'remediationtype', 'remediationend', 'postimpactassessmentdate', 'finallabresultsdate'], dtype='object')

In [96]:
dfbi =  df3[['incidentdate','company','contaminant','estimatedquantity','spillareahabitat','cause']]

,id,status,company,incidentnumber,incidentdate,contaminant,estimatedquantity,sitelocationname,spillareahabitat,lastupdatedby,cause,updatefor,latitude,longitude,statesaffected,attachments,jivdate,lga,zonaloffice,certificatedate,certificatenumber,formadate,reportdate,spillstopdate,initialcontainmentmeasures,descriptionofimpact,formbdate,formcdate,jivpresent,typeoffacility,cleanupcompleteddate,cleanupmethods,cleanupdate,quantityrecovered,postcleanupinspectiondate,finalsamplingdate,estimatedspillarea,remediationstart,remediationtype,remediationend,postimpactassessmentdate,finallabresultsdate
4936,5086,confirmed,SEPLAT,SEPLAT/WA/OS/05/2020,2020-02-06,ch,NaN,SAPELE WELL 33/34/35 (DRILLING RIG).,la,"NOSDRA, 2014-08-23 WA003, 2019-04-23 PH002, 20...",sab,5086.0,5.8939167,5.5975278,DE,"[{""url"":""data/attachments/5086/IMG_20200210_13...",2020-02-08,Sapele,wa,NaN,NaN,NaN,2020-02-06,NaN,bm,"Soil, vegetation and swamp were impacted, the...",NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",rg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4944,5094,confirmed,SPDC,2066951,2018-04-26,cr,580.0,24'' TRANS RAMOS PIPELINE (OIL) @ ODIMODI.,sw,"NOSDRA, 2014-08-22 WA003, 2019-04-23 PH002, 20...",cor,5094.0,5.2519722,5.3797222,DE,"[{""url"":""data/attachments/5094/SPDC B1.jpg"",""c...",2018-05-06,Burutu,wa,NaN,NaN,NaN,2018-05-02,NaN,bm,"Vegetation and economic trees were impacted, t...",NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",pl,2020-03-30,Manual.,2020-02-11,212,2021-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4945,5095,confirmed,SPDC,2067178,2018-04-27,cr,0.8,24'' TRANS RAMOS PIPELINE (OIL) @ ODIMODI.,sw,"NOSDRA, 2014-06-18 WA003, 2019-04-23 PH002, 20...",cor,5095.0,5.1889722,5.3957222,DE,"[{""url"":""data/attachments/5095/SPDC A1.jpg"",""c...",2018-05-05,Burutu,wa,NaN,NaN,NaN,2018-05-02,NaN,bm,"Vegetation was impacted, the spill was within ...",NaN,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",pl,2020-03-04,Manual.,2020-02-10,NaN,2021-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5572,5760,confirmed,NAOC,2020/LAR/012/022,2020-02-05,cr,1.0,10'' IRRI - KWALE PIPELINE AT OFAGBE,la,"NOSDRA, 2019-06-10 WA003, 2020-02-12 WA003, 20...",sab,5760.0,5.5486111,6.3680556,DE,"[{""url"":""data/attachments/5760/NAOC A (1).jpg""...",2020-02-07,Isoko-North,wa,NaN,NaN,NaN,2020-02-06,2020-02-06,other: Patch welding,"Vegetation and soil were impacted, the spill w...",2020-03-03,NaN,"nosdra,dpr,mnstry,cmpny,cmmnt",pl,NaN,NaN,NaN,NaN,2022-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7548,8888,confirmed,SEPLAT,SEPLAT/WA/OS/01/2020,2020-01-08,cr,35.0,SAPELE WELL 24 4'' FLOW LINE @ SAPELE OKPE.,la,"2014-02-21 AB008, 2020-01-09 WA003, 2020-01-17...",cor,8888.0,5.8896667,5.6128333,DE,"[{""url"":""data/attachments/8888/SEPLAT JIV PIC ...",2020-01-09,Sapele,wa,NaN,NaN,NaN,2020-01-08,NaN,other:concrete culvert,"Vegetation and soil were impacted, the spill w...",NaN,2020-02-06,"nosdra,dpr,mnstry,cmpny,cmmnt",fl,2020-03-19,Manual.,2020-03-18,31.4,2021-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df4w = df3.groupby([pd.Grouper(key='incidentdate', freq='W')])['estimatedquantity'].agg(['sum','size'])
df4m = df3.groupby([pd.Grouper(key='incidentdate', freq='M')])['estimatedquantity'].agg(['sum','size'])

In [95]:
df4m.tail()

,sum,size
incidentdate,,
2022-08-31,863.26954,67
2022-09-30,493.28310,60
2022-10-31,538.75750,40
2022-11-30,1240.31300,19
2022-12-31,450.96000,14
